In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,confusion_matrix, f1_score, precision_score, recall_score

from keras.models import Model
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
import warnings
warnings.filterwarnings('ignore')




Using TensorFlow backend.


In [24]:
# eda_columns=['article_id', 'article_id_flag', 'clean', 'clean_comment_text',
#        'comment_text', 'count_ip', 'count_links', 'id', 'identity_hate',
#        'insult', 'ip', 'label_num', 'link', 'num_char', 'num_char_norm',
#        'num_words', 'num_words_norm', 'obscene', 'punctuation',
#        'punctuation_norm', 'sent_num', 'sent_num_norm', 'severe_toxic',
#        'stop_words', 'stop_words_norm', 'threat', 'title', 'title_norm',
#        'toxic', 'unique', 'unique_norm', 'upper', 'upper_norm', 'username',
#        'username_nums']
# non_eda_columns=['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
#        'insult', 'identity_hate', 'clean', 'label_num', 'sent_num',
#        'sent_num_norm', 'num_words', 'num_words_norm', 'num_char',
#        'num_char_norm', 'stop_words', 'stop_words_norm', 'punctuation',
#        'punctuation_norm', 'upper', 'upper_norm', 'title', 'title_norm',
#        'unique', 'unique_norm', 'ip', 'count_ip', 'link', 'count_links',
#        'article_id', 'article_id_flag', 'username', 'username_nums',
#        'clean_comment_text']
# test_columns=['id', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
#        'identity_hate', 'comment_text', 'sent_num', 'sent_num_norm',
#        'num_words', 'num_words_norm', 'num_char', 'num_char_norm',
#        'stop_words', 'stop_words_norm', 'punctuation', 'punctuation_norm',
#        'upper', 'upper_norm', 'title', 'title_norm', 'unique', 'unique_norm',
#        'ip', 'count_ip', 'link', 'count_links', 'article_id',
#        'article_id_flag', 'username', 'username_nums', 'clean_comment_text']

# train=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/non_eda_train.txt",names=non_eda_columns,header=0)
# eda_train=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/eda_train.txt",names=eda_columns,header=0)
# test=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_test_labels.txt",names=test_columns,header=0)

# # train=train.head(1000)
# # test=test.head(1000)
c_t=['clean', 'clean_comment_text', 'id', 'toxic', 'severe_toxic', 'obscene',
       'threat', 'insult', 'identity_hate', 'comment_text', 'sent_num',
       'sent_num_norm', 'num_words', 'num_words_norm', 'num_char',
       'num_char_norm', 'stop_words', 'stop_words_norm', 'punctuation',
       'punctuation_norm', 'upper', 'upper_norm', 'title', 'title_norm',
       'unique', 'unique_norm', 'ip', 'count_ip', 'link', 'count_links',
       'article_id', 'article_id_flag', 'username', 'username_nums',
       'clean_comment_text_y']
test=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_eda_test.txt",names=c_t,header=0)

c=['clean_x', 'clean_comment_text', 'id', 'comment_text', 'toxic',
       'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate',
       'clean_y', 'label_num', 'sent_num', 'sent_num_norm', 'num_words',
       'num_words_norm', 'num_char', 'num_char_norm', 'stop_words',
       'stop_words_norm', 'punctuation', 'punctuation_norm', 'upper',
       'upper_norm', 'title', 'title_norm', 'unique', 'unique_norm', 'ip',
       'count_ip', 'link', 'count_links', 'article_id', 'article_id_flag',
       'username', 'username_nums', 'clean_comment_text_y']
train=pd.read_csv("/Users/jixusheng/Documents/kaggle/eda_nlp-master/new_eda_train.txt",names=c,header=0)

In [25]:
embedding_file="/Users/jixusheng/Documents/kaggle/glove.6B.50d.txt"
#embedding_file="/Users/jixusheng/Documents/kaggle/crawl-300d-2M.vec"
#embedding_file="/Users/jixusheng/Documents/kaggle/GoogleNews-vectors-negative300.bin"

In [26]:
# eda_train.columns

In [27]:
 train.head(3)

clean_x                                 clean_comment_text  \
0        1  explanationwhy the edit make under my username...   
1        1  explanationwhy the edit make under my username...   
2        1  explanationwhy the edit make under my username...   

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
2  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   

   severe_toxic  obscene  threat  insult  identity_hate  ...  unique_norm  \
0             0        0       0       0              0  ...      0.04908   
1             0        0       0       0              0  ...      0.04908   
2             0        0       0       0              0  ...      0.04908   

                 ip  count_ip  link  count_links  article_id  article_id_flag  \
0  ['89.205.38.27']         1    []            0          []                0   
1  ['89.205.38.27']         1    []            0          []                0   
2  ['89.205.38.27']         1    []            0          []                0   

   username  username_nums                               clean_comment_text_y  
0        []              0  explanationwhy edit make username hardcore met...  
1        []              0  explanationwhy edit make username hardcore met...  
2        []              0  explanationwhy edit make username hardcore met...  

[3 rows x 37 columns]

In [28]:
test.head(3)


clean                                 clean_comment_text                id  \
0      1  not you for understand i think very highly of ...  0001ea8717f6de06   
1      1  thank you for understand i think very highly o...  0001ea8717f6de06   
2      1  thank you for understand i think very highly o...  0001ea8717f6de06   

   toxic  severe_toxic  obscene  threat  insult  identity_hate  \
0      0             0        0       0       0              0   
1      0             0        0       0       0              0   
2      0             0        0       0       0              0   

                                        comment_text  ...  unique_norm  ip  \
0  Thank you for understanding. I think very high...  ...     0.017073  []   
1  Thank you for understanding. I think very high...  ...     0.017073  []   
2  Thank you for understanding. I think very high...  ...     0.017073  []   

   count_ip  link  count_links  article_id  article_id_flag  username  \
0         0    []            0          []                0        []   
1         0    []            0          []                0        []   
2         0    []            0          []                0        []   

   username_nums                               clean_comment_text_y  
0              0  thank understand think highly would revert wit...  
1              0  thank understand think highly would revert wit...  
2              0  thank understand think highly would revert wit...  

[3 rows x 35 columns]

In [29]:
#input1_insult_larger[['insult','clean_comment_text','id']].to_csv('/Users/jixusheng/Documents/kaggle/eda_nlp-master/input_larger_100.txt', sep='\t', index=False)


In [30]:
X_train=train["clean_comment_text"].fillna("fillna").values
#X_train=eda_train["clean_comment_text"].fillna("fillna").values

In [31]:
y_train =train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
#y_train =eda_train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [32]:
X_test = test["clean_comment_text"].fillna("fillna").values

In [33]:
y_test = test[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values

In [34]:
#max_features = 100
max_features = 100000
maxlen = 200
###for google news and fasttext
#embed_size = 300
###for glove
embed_size = 50
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)



In [35]:
###glove and fast text
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(embedding_file))

####word2vec
# import gensim.models.keyedvectors as word2vec
# word2vecDict = word2vec.KeyedVectors.load_word2vec_format(embedding_file, binary=True)
# embeddings_index = dict()
# for word in word2vecDict.wv.vocab:
#             embeddings_index[word] = word2vecDict.word_vec(word)

In [36]:
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [37]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred,average='macro')
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))
#             self.y_val=np.argmax(self.y_val, axis=1)
#             y_pred=np.argmax(y_pred, axis=1)
#             r_score = recall_score(self.y_val, y_pred,average='macro')
#             print("\n Recall - epoch: %d - score: %.6f \n" % (epoch+1, r_score))
#             p_score = precision_score(self.y_val, y_pred,average='macro')
#             print("\n Presion - epoch: %d - score: %.6f \n" % (epoch+1, r_score))
#             r_score = f1_score(self.y_val, y_pred,average='macro')
#             print("\n f1 score - epoch: %d - score: %.6f \n" % (epoch+1, r_score))

In [38]:
# class Metrics(Callback):
#     def on_train_begin(self, logs={}):
#         self.val_f1s = []
#         self.val_recalls = []
#         self.val_precisions = []

#     def on_epoch_end(self, epoch, logs={}):
#         val_predict = np.argmax(np.asarray(self.model.predict(self.validation_data[0])), axis=1)
#         val_targ = np.argmax(self.validation_data[1], axis=1)
#         _val_recall = recall_score(val_targ, val_predict)
#         _val_precision = precision_score(val_targ, val_predict)
#         _val_f1 = f1_score(val_targ, val_predict, average='macro')
#         self.val_f1s.append(_val_f1)
#         print(' — val_f1:' ,_val_f1)
#         print(' — val_precision:',_val_precision)
#         print(' — val_recall:',_val_recall)
#         return

# metrics = Metrics()


In [39]:
filter_sizes = [1,2,3,5]
num_filters = 32

In [40]:
def get_model():    
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.4)(x)
    x = Reshape((maxlen, embed_size, 1))(x)
    
    conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embed_size), kernel_initializer='normal',
                                                                                    activation='elu')(x)
    conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embed_size), kernel_initializer='normal',
                                                                                    activation='elu')(x)
    conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embed_size), kernel_initializer='normal',
                                                                                    activation='elu')(x)
    conv_3 = Conv2D(num_filters, kernel_size=(filter_sizes[3], embed_size), kernel_initializer='normal',
                                                                                    activation='elu')(x)
    
    maxpool_0 = MaxPool2D(pool_size=(maxlen - filter_sizes[0] + 1, 1))(conv_0)
    maxpool_1 = MaxPool2D(pool_size=(maxlen - filter_sizes[1] + 1, 1))(conv_1)
    maxpool_2 = MaxPool2D(pool_size=(maxlen - filter_sizes[2] + 1, 1))(conv_2)
    maxpool_3 = MaxPool2D(pool_size=(maxlen - filter_sizes[3] + 1, 1))(conv_3)
        
    z = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2, maxpool_3])   
    z = Flatten()(z)
    z = Dropout(0.1)(z)
        
    outp = Dense(6, activation="sigmoid")(z)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [41]:
model = get_model()

In [42]:
batch_size = 256
epochs = 3

In [43]:
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)

# X_tra=x_train
# y_tra=y_train
# X_val=x_test
# y_val=y_test
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

In [44]:
hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=2)


Train on 757929 samples, validate on 39891 samples
Epoch 1/3
 - 2025s - loss: 0.0670 - accuracy: 0.9772 - val_loss: 0.0477 - val_accuracy: 0.9827

 ROC-AUC - epoch: 1 - score: 0.983540 

Epoch 2/3
 - 2035s - loss: 0.0508 - accuracy: 0.9815 - val_loss: 0.0454 - val_accuracy: 0.9832

 ROC-AUC - epoch: 2 - score: 0.987137 

Epoch 3/3
 - 2036s - loss: 0.0487 - accuracy: 0.9820 - val_loss: 0.0420 - val_accuracy: 0.9842

 ROC-AUC - epoch: 3 - score: 0.988668 



In [45]:
#pre = model.predict(x_test)